In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [2]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from typing import Sequence
from langchain_core.messages import HumanMessage

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class Trimmer:
    def invoke(self, messages):
        return messages[-1:]

trimmer = Trimmer()


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

async def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    print(f"Messages after trimming: {len(trimmed_messages)}")
    for msg in trimmed_messages:
        print("User asked:", msg.content)
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = await model.ainvoke(prompt) 
    return {"messages": state["messages"] + [response]}

workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

config = {"configurable": {"thread_id": "abc123"}}

In [13]:
import asyncio

async def chat():
    print("Chatbot ready! Type 'exit' to quit.\n")
    messages = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        messages.append(HumanMessage(content=user_input))
        output = await app.ainvoke(
            {"messages": messages, "language": "English"},
            config=config
        )
        ai_reply = output["messages"][-1]
        print(f"Bot: {ai_reply.content}\n")
        messages.append(ai_reply)  # Add bot reply to history

await chat()

Chatbot ready! Type 'exit' to quit.

Messages after trimming: 1
User asked: who is Thor?
Bot: Thor is a prominent figure with two main interpretations:

1.  **In Norse Mythology (The Original):**
    *   **God of Thunder:** Thor is one of the most powerful and well-known gods in the Norse pantheon. He is the god of thunder, lightning, storms, strength, oak trees, and the protection of mankind.
    *   **Son of Odin:** He is the son of Odin (the Allfather) and Jörð (the personification of the Earth).
    *   **Protector of Asgard and Midgard:** Thor is a fierce protector of Asgard (the realm of the gods) and Midgard (the realm of humans) from their enemies, primarily the Jötnar (giants).
    *   **Mjolnir:** His most famous attribute is his enchanted hammer, **Mjolnir**, which can level mountains, return to his hand after being thrown, and only be wielded by those worthy. He also possesses a belt of strength (Megingjörð) and iron gloves (Járngreipr).
    *   **Personality:** He is often